Configuration

In [ ]:
import torch

DEVICE = torch.device('cuda:0')
EPOCHS = 2000
LEARNING_RATE = 0.0003
BATCH_SIZE = 16

Create model

In [ ]:
import torch.nn as nn

class Model(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.rnn = nn.LSTM(input_size=3, hidden_size=128, num_layers=2, bias=False, batch_first=True)
        self.linear1 = nn.Linear(128, 64, bias=True)
        self.linear2 = nn.Linear(64, 1, bias=True)
        self.norm = nn.Sigmoid()

    def forward(self, x):
        x = self.rnn(x)[0][:,-1,:]
        x = self.linear1(x)
        x = self.linear2(x)
        out = self.norm(x)
        return out

Prepare dataset

In [ ]:
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np


class MyDataset(Dataset):

    def __init__(self, raw_data, device='cpu') -> None:
        super().__init__()
        self.raw_data = raw_data
        self.device = device

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, index):
        x = torch.tensor(self.raw_data[index, :, :-1]).float().to(self.device)
        y = torch.tensor(self.raw_data[index, :, -1][0]).float().unsqueeze(-1).to(self.device)
        return x, y


raw_data = np.load('dataset.npy')
dataset = MyDataset(raw_data, device=DEVICE)
train_set, validation_set = random_split(dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_set, batch_size=len(validation_set))

Train

In [ ]:
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinaryPrecision, BinaryRecall
from torchmetrics.functional.classification import binary_accuracy, binary_precision, binary_recall, binary_f1_score
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

@torch.no_grad()
def validate(model, dataloader):
    model.eval()
    x, y = next(iter(dataloader))
    preds = model(x)
    loss = loss_fn(preds, y)
    accuracy = binary_accuracy(preds.cpu(), y.cpu())
    precision = binary_precision(preds.cpu(), y.cpu())
    recall = binary_recall(preds.cpu(), y.cpu())
    f1 = binary_f1_score(preds.cpu(), y.cpu())
    return loss.cpu().item(), accuracy, precision, recall, f1

model = Model().to(DEVICE)
loss_fn = nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

metric_accuracy = BinaryAccuracy().to(DEVICE)
metric_precision = BinaryPrecision().to(DEVICE)
metric_recall = BinaryRecall().to(DEVICE)
metric_f1 = BinaryF1Score().to(DEVICE)

writer = SummaryWriter()

for i in tqdm(range(EPOCHS), desc='Training epochs'):
    model.train()
    ep_loss = []
    for mb_x, mb_y in train_dataloader:
        optim.zero_grad()
        out = model(mb_x)
        loss = loss_fn(out, mb_y)
        loss.backward()
        optim.step()

        ep_loss.append(loss.cpu().item())
        metric_accuracy(out, mb_y)
        metric_f1(out, mb_y)
        metric_precision(out, mb_y)
        metric_recall(out, mb_y)

    if (i + 1) % 100 == 0:
        torch.save(model.state_dict(), f"./models/{i + 1}_model.pt")

    v_loss, v_acc, v_pre, v_rec, v_f1 = validate(model, validation_dataloader)
    writer.add_scalars('Loss', {'train': np.mean(ep_loss), 'validation': v_loss}, i)
    writer.add_scalars('Metric/Accuracy', {'train': metric_accuracy.compute(), 'validation': v_acc}, i)
    writer.add_scalars('Metric/Precision', {'train': metric_precision.compute(), 'validation': v_pre}, i)
    writer.add_scalars('Metric/Recall', {'train': metric_recall.compute(), 'validation': v_rec}, i)
    writer.add_scalars('Metric/F1', {'train': metric_f1.compute(), 'validation': v_f1}, i)